### hihello에서 마지막 o를 추측하는 예제

#### 1. 데이터 초기화

In [7]:
import tensorflow as tf
import numpy as np

# 1) 학습 데이터 "hihello" 에서 중복되지 않는 문자를 숫자로 표현한다.
idx2char = ['h', 'i', 'e', 'l', 'o'] # h = 0, i = 1, e = 2, L = 3, o = 4

x_data = [[0, 1, 0, 2, 3, 3]] # hihell

# 2) 숫자로 표현된 데이터를 one-hot encoding 한다.
x_one_hot = [[[1, 0, 0, 0, 0],  # h = 0
              [0, 1, 0, 0, 0],  # i = 1
              [1, 0, 0, 0, 0],  # h = 0
              [0, 0, 1, 0, 0],  # e = 2
              [0, 0, 0, 1, 0],  # l = 3
              [0, 0, 0, 1, 0]]] # l = 3

# 3) 답안으로 사용할 데이터를 생성한다.
y_data = [[1, 0, 2, 3, 3, 4]] # ihello

print(y_data)

[[1, 0, 2, 3, 3, 4]]


#### 2. 파라미터 초기화

In [8]:
num_claases = 5      # 정답 크기 (one-hot으로 나타내는 크기)
input_dim = 5        # one-hot size, 입력값은 0부터 4까지 총 5개.
hidden_size = 5      # output size, one-hot size와 같음.
sequence_length = 6  # 입력으로 들어가는 문장 길이, ihello = 6
batch_size = 1       # 한문장씩 배치. (학습에 사용되는 문자가 하나밖에 없기때문)
learning_rate = 0.1  # 학습율

# placeholder 선언 (선언과 동시에 초기화 하는 것이 아니라 선언 후, 값을 전달한다.)
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])

AttributeError: ignored

#### 3. BasicRNNCell 실행

In [ ]:
# 은닉층에서 표현되는 one-hot 값을 전달한다. (hidden_size = 5)
# 기본적인 RNNCell을 생성한다.
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

# cell을 0으로 초기화 한다.
initial_state = basic_cell.zero_state(batch_size, tf.float32)

#### 4. dynamic_rnn 실행

In [ ]:
# RNNCell로 지정된 반복 신경망을 만든다.
# 내부적으로 __call__() 함수를 호출하여 Time Step 마다 Cell의 복사본을 만들고 서로 연결한다.
outputs, _states = tf.nn.dynamic_rnn(basic_cell, X, initial_state=initial_state, dtype=tf.float32)

# 모든 요소가 1로 설정된 Tensor를 생성한다.
weights = tf.ones([batch_size, sequence_length])

#### 5. Cross Entropy 생성

In [ ]:
# RNNCell에 대한 Cross Entropy 생성.
# 예측값을 넣어준다. 예측값과 실제값이 얼마나 차이나는지 계산
seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

#### 6. Optimizer 실행

In [ ]:
# Tensor의 차원에서 요소의 평균 계산.
loss = tf.reduce_mean(seq_loss)

# Adam 알고리즘을 이용한 최적화 (확률론적 최적화 방법)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#### 7. 학습 실행

In [ ]:
# tf.argmax(a, 0), 1차원 배열에서 가장 큰 값을 찾아 인덱스를 리턴.
y = prediction = tf.argmax(outputs, axis=2)

# tf.Session(), 파이썬 객체와 데이터, 객체의 메모리가 할당되어 있는 실행 환경 사이를 연결.
with tf.Session() as sess:
    
    # Tensor Flow 변수 초기화 
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        
        # 학습 연산 수행
        loss_val, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(y, feed_dict={X: x_one_hot})
        
        if step % 400 ==0:
            print("step = ", step, ", loss = ", loss_val, ", prediction = ", result, ", target = ", y_data)

            # 예측된 문자열
            result_str = [idx2char[c] for c in np.squeeze(result)]
            
            print("\tPrediction = ", ''.join(result_str))